# ⏱️ Vizualizace časových řad

V tomto Jupyter Notebooku si ukážeme práci s časovými řadami a jejich vizualizaci. Jako **časovou řadu** vnímáme soubor měření nějaké náhodné veličiny, která se v čase vyvíjí -> proces. 

Značíme jej například takto:

$$\large Y = (y_1, y_2, y_3, \ldots , y_{t-1}, y_t)$$

Časové řady mohou mít zajímavé vlastnosti, které nám mohou pomoci při analýze daného procesu. Můžete je potkat všude - akcie, covid-19, ceny paliv, počasí apod.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px

import statsmodels
import statsmodels.api as sm

from pandas import DataFrame
from pandas import Grouper
import matplotlib

matplotlib.rcParams["figure.figsize"] = (12, 8)

sns.set_theme(style="darkgrid", palette=sns.color_palette("deep"))
pd.options.plotting.backend = "plotly"

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import ipywidgets.widgets as widgets

## 💽 Nahrání dat
Práci s časovými řadami si ukážeme na datasetu [**Airline Passengers**](https://github.com/jbrownlee/Datasets) ✈️. Jedná se o velmi přímočarý dataset, který obsahuje pouze dva příznaky - časový údaj (měsíc a rok, _Month_) a počet cestujících leteckou dopravou (_Passengers_). Pojďme si jej načíst pomocí `pandas`. 🐼

In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv"
)

In [ ]:
df

In [ ]:
df["Passengers"].plot(title="Airline passengers")

Na grafu vidíme měsíční vývoj v počtu pasažérů v letecké dopravě v letech `1949-1960`. 

Jediné, co z grafu nevyčteme, je právě časová osa, kterou máme zde diskrétní (`0-t`, osa x). Takto ale přicházíme o cennou informaci, kterou v datasetu máme. 🙈

⛔️ Pozn. k datasetu: V praxi se s takto čistou časovou řadou se zjevným trendem setkáte jen velmi zřídka. Není proto náhodou, že tento dataset často potkáte v tutoriálech. 🙈

### ⌛ Časový index
`Pandas` má nativní podporu pro [časový index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html). Díky tomu můžeme datumy lépe analyzovat (a např. tak zjistit, jestli anomální chování odpovídá nějakému státnímu svátku) a pracovat s daty v nějakém časovém intervalu - hodina, týden, měsíc apod.

In [ ]:
df.info()

In [ ]:
df["Date"] = pd.to_datetime(df["Month"])  # je třeba převést string na datum
df = df.set_index("Date")["Passengers"]

In [ ]:
df

In [ ]:
df.plot(title="Monthly airline passengers")

Graf už teď efektivně zobrazuje celý dataset. Z časové řady však můžeme vytěžit více.

### 📈📉📈 Rozpad grafu
Pojďme si vykreslit **každý rok zvlášť**. Vykreslíme vždy hodnoty pro jednotlivé roky po měsících a barevně odlišíme aktuální rok. 

In [ ]:
# https://seaborn.pydata.org/examples/timeseries_facets.html

sns.set_theme(style="dark")
flights = sns.load_dataset("flights")

# Plot each year's time series in its own facet
g = sns.relplot(
    data=flights,
    x="month",
    y="passengers",
    col="year",
    hue="year",
    kind="line",
    palette="crest",
    linewidth=4,
    zorder=5,
    col_wrap=3,
    height=2,
    aspect=1.5,
    legend=False,
)

# Iterate over each subplot to customize further
for year, ax in g.axes_dict.items():

    # Add the title as an annotation within the plot
    ax.text(0.8, 0.85, year, transform=ax.transAxes, fontweight="bold")

    # Plot every year's time series in the background
    sns.lineplot(
        data=flights,
        x="month",
        y="passengers",
        units="year",
        estimator=None,
        color=".7",
        linewidth=1,
        ax=ax,
    )

# Reduce the frequency of the x axis ticks
ax.set_xticks(ax.get_xticks()[::2])

# Tweak the supporting aspects of the plot
g.set_titles("")
g.set_axis_labels("", "Passengers")
g.tight_layout()

### 🗺️ Heatmapa
Ve stejném duchu můžeme použít i heatmapu.

In [ ]:
years = pd.DataFrame(
    {name.year: group.values for name, group in df.groupby(pd.Grouper(freq="A"))}
)
years.index += 1

In [ ]:
years

In [ ]:
fig = years.T.plot(kind="imshow", title="Heatmap of flight passengers")
#fig.update_layout(xaxis=dict(ticks))
fig.update_xaxes(tick0=0, dtick=1, title="Month")
fig.update_yaxes(title="Year")

### 🗺️ Heatmap3D / 3D Surface 
Heatmapu můžete použít i 3D a můžeme tak zdůraznit rozdíly v hodnotách. Čitelnost však posuďte sami.☝️👀

In [ ]:
# interactive plotting
%matplotlib notebook 

X = []
Y = []
Z = []

for row, cols in years.iterrows():
    for col in cols.index:
        X.append(row)
        Y.append(col + 1)
        Z.append(cols[col])

fig = plt.figure(figsize=(9, 6))
ax = fig.add_subplot(111, projection="3d")
ax.plot_trisurf(X, Y, Z, cmap=plt.cm.Spectral_r)
plt.show()

In [ ]:
# renders the figure in a notebook 
%matplotlib inline 

### 🐻‍❄️ Polární souřadnice
Můžeme sestrojit [polární graf](https://matplotlib.org/stable/gallery/pie_and_polar_charts/polar_demo.html) (angl. polar plot) pomocí [polárních souřadnic](https://cs.wikipedia.org/wiki/Polárn%C3%AD_graf), ve kterém si hodnoty vykreslíme postupně proti směru hodinových ručiček.

Polární graf se hodí k porovnání více řad dat, nicméně může být obtížně čitelný. Pokud by vás zajímal více, doporučuji prozkoumat balíček [Plotly](https://plotly.com/python/polar-chart/), kde najdete celou řadu různých variant polárního grafu. 🕵🏻‍♂️

In [ ]:
import matplotlib.colors as mcolors

In [ ]:
monthly = df.resample("m").mean() # gets months from datetime-like index and averages them

r = monthly # distance (= radius)
theta = 2 * np.pi * ((monthly.index.month - 1) / 12) # angle at which we want to draw the curve

fig, ax = plt.subplots(1, 1, subplot_kw=dict(projection="polar"), figsize=(12, 7))
ax.plot(theta, r)
ax.set_rticks([100, 300, 600]) # less radial ticks
ax.set_rlabel_position(-22.5) # move radial labels away from plotted line

ax.xaxis.set_ticks([2 * np.pi * x / 12 for x in range(1, 13)]) # how often angles labels occur
ax.set_thetagrids([30 * i for i in range(12)], labels=list(range(1, 13))) # rename the angles labels
ax.grid(True)
ax.set_title("A line plot on a polar axis of airline passengers", va='bottom')

plt.show()

### 📊 Statistiky

I k časovým řadám jsme schopni si spočítat základní popisné statistiky. Pojďme využít nám už známé funkce `pd.DataFrame.describe()`.

In [ ]:
df.describe()

 Dále se podíváme na histogram hodnot.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
df.hist(backend="matplotlib", ax=ax)
ax.set_title("Histogram of airline passengers")
plt.show()

Statistiky i histogram nám sice dají nějakou představu o datech, ale u časových řad to nemusí být vždy postačující. Často v časově závislých datech dochází k dlouhodobém růstu či poklesu, čemuž se říká **trend**. Přítomnost trendu způsobuje, že většina těchto statistik bude oproti hodnotám některých intervalů vychýlena, jelikož dochází ke změně v distribuci. Například v histogramu máme relativně malé četnosti u výskytu "600 pasažérů za měsíc". Toto číslo bylo v roce 1960 považované za rekordní, avšak v poměru s dnešními počty cestujících by se jednalo o velmi špatný měsíc.

## ✈️ Lag plot

Dalším vizualizačním nástrojem je tzv. **graf zpoždění** (angl. [`lag plot`](https://pandas.pydata.org/docs/reference/api/pandas.plotting.lag_plot.html)), který nám může pomoct najít nějaký vztah mezi časovou řadou ($y_t$) a její zpožděnou verzí ($y_{t+k}$). _Lag_ tedy představuje míru časového zpoždění jednoho bodu časové řady za jiným bodem.

Můžeme tak odhalit různé vzorce v datech jako např. sezónní závislosti, trendy, náhodnost, outliery apod.

Graf zpoždění se vykreslí tak, že na ose x jsou znázorněna data časové řady a na ose y jednotlivá zpoždění bodů časové řady. Pokud je pro datový bod zpoždění (_lag_) rovno jedné, je zpožděním předchozí datový bod. Pokud je zpoždění rovno dvěma, je zpožděním datový bod předcházející dvěma datovým bodům v čase.

In [ ]:
# original graph and lag plot
fig, ax = plt.subplots(1, 2, figsize=(12, 7))
ax[0].plot(df)
ax[0].set_xlabel('Datetime')
ax[0].set_ylabel('Passengers')
pd.plotting.lag_plot(df, lag=1, c="b", label="data point", ax=ax[1])
plt.show()

In [ ]:
def lag_plot(lag):

    fig = plt.figure(figsize=(10, 5))

    ax = fig.add_subplot(111)
    pd.plotting.lag_plot(df, lag=lag, c="b", label="data point", ax=ax)
    ax.plot([100, 650], [100, 650], c="g", label="y=x")
    ax.legend()

    fig.suptitle("Lag plot")
    plt.show()
widgets.interact(lag_plot, lag=widgets.IntSlider(1, 1,16));

> <span style="font-size:200%;"> 🤔 </span> Jelikož víme, že data jsou měřená měsíčně, najdete v grafu pro lag=12 nějakou závislost?

Pro zajímavost uvádíme citaci několika interpretací lag plotu z [GeeksForGeeks](https://www.geeksforgeeks.org/lag-plots/):

- **Distribution of Model**: Distribution of model here means deciding what is the shape of data on the basis of the lag plot. Below are some examples of lag plot and their original plot:
    - If the lag plot is linear, then the underlying structure is of the autoregressive model.
    - If the lag plot is of elliptical shape, then the underlying structure represents a continuous periodic function such as sine, cosine, etc.
- **Outliers**: Outliers are a set of data points that represent the extreme values in the distribution
- **Randomness in data**: The lag plot is also useful for checking whether the given dataset is random or not. If there is randomness in the data then it will be reflected in the lag plot, if there is no pattern in the lag plot.
- **Seasonality**: If there is seasonality in the plot then, it will give a periodic lag plot.
- **Autocorrelation**: If the lag plot gives a linear plot, then it means the autocorrelation is present in the data, whether there is positive autocorrelation or negative that depends upon the slope of the line of the dataset. If more data is concentrated on the diagonal in lag plot, it means there is a strong autocorrelation.

## 🐈‍⬛🐈 Diferencování

Pomocí **diferencování** můžeme "očistit" časovou řadu o její zpožděnou verzi, tedy 


$$Y_t' = Y_t - Y_{t-k} $$

Diferencování může pomoci stabilizovat střední hodnotu časové řady tím, že odstraní změny v časové úrovni řady (tj. časovou závislost) a tím eliminuje/snižuje trend a sezónnost.

In [ ]:
def diff_plot(k):

    fig = plt.figure(figsize=(10, 5))
    diff = df.diff(k) # num of differences

    ax = fig.add_subplot(111)
    ax.plot(diff.index, diff, label=f"k={k}")
    ax.legend()

    fig.suptitle("Difference plot")
    plt.show()


widgets.interact(diff_plot, k=widgets.IntSlider(1, 1, 16));

> <span style="font-size:200%;"> 🤔 </span> Jak to vypadá s k=12?

## 🔩 Dekompozice

Některé časové řady můžeme rozložit na podsložky - **trend**, **sezónnost** a **rezidua**. Tyto rozklady mohou být v 

`aditivní`

$$Y_t = T_t + S_t + \epsilon_t$$

nebo `multiplikativní`
$$Y_t = T_t \cdot S_t \cdot \epsilon_t$$
 formě.


Z modulu `statsmodels.tsa` použijeme funkci [`seasonal_decompose()`](https://www.statsmodels.org/dev/generated/statsmodels.tsa.seasonal.seasonal_decompose.html), která provede dekompozici na trend ($T$), Sezónost ($S$) a rezidua ($\epsilon$), ideálně [IID](https://en.wikipedia.org/wiki/Independent_and_identically_distributed_random_variables) (tj. independent and identically distributed).

In [ ]:
aditive_decompose = statsmodels.tsa.seasonal.seasonal_decompose(df, model="aditive", period=12)

fig = aditive_decompose.plot()
fig.suptitle('Aditive decomposition', y=1.025)
plt.show()

In [ ]:
multip_decompose = statsmodels.tsa.seasonal.seasonal_decompose(df, model="multiplicative", period=12)

fig = multip_decompose.plot()
fig.suptitle('Multiplicative decomposition', y=1.025)
plt.show()

 📝 Dostali jsme dva rozklady, ale který je lepší? Pokud jsou rezidua a sezónnost nějakým způsobem ovlivněná trendem, což se také projevuje tím, že se hodnoty v jednotlivých sezónách časové řady monotónně zvětšují/zmenšují v čase, je lepší zvolit multiplikativní dekompozici. Pokud jsou sezónnost a reziduální složky nezávislé na trendu, jedná se o aditivní řadu. Více si můžete přečíst např. [zde](https://dziganto.github.io/python/time%20series/Introduction-to-Time-Series/).

## 🪟 Klouzavá okénka

**Klouzavá okénka** (angl. rolling window) mají u časových řad mnohá využití. Dají se například použít k vyhlazení časových řad nebo k počítání klouzavého průměru, klouzavého rozptylu nebo jiné klouzavé agregace. K tomu všemu použijeme metodu [`rolling`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html), které specifikujeme velikost okénka a zda bude okénko centrované.

Velmi zjednodušeně řečeno: Vezmeme okénko o velikosti _k_ (tzn. _k_ po sobě jdoucích hodnot v daném čase) a provedeme s ním nějakou požadovanou matematickou operaci.

![](img/window.png) [zdroj obrázku](https://towardsdatascience.com/rolling-windows-in-numpy-the-backbone-of-time-series-analytical-methods-bc2f79ba82d2)

In [ ]:
df.rolling(12, center=True).agg({"mean", "median", "min", "max"}).plot(
    kind="line", title="Aggregate functions on windows")

### 📦 Box plot
Tyto statistiky můžeme vizualizovat i jiným způsobem, např. krabicovým grafem.

In [ ]:
years.boxplot(title="Boxplot of number of air passengers")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(df.rolling(12, center=True).mean(), label="12M window mean")
ax.plot(aditive_decompose.trend, label="Aditive decompose trend")
ax.legend()
ax.set_title("Srovnání")
plt.show()

📝 Všimněte si, že když vykreslíme klouzavý průměr s periodou 12 a srovnáme jej s trendem z dekompozice, tak jsou téměř identické. ☝️

### ⬇️ Downsampling
S časovým indexem můžeme "downsamplovat" (tj. podvzorkovat) data na menší frekvenci a to pomocí funkce [resample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html). Jednoduše řečeno, metoda zachovává pouze každých několik záznamů, čímž vytváří aproximaci signálu na nižší frekvenci. Ještě jednodušeji se jedná o agregaci dat pomocí nové časové periody. 

Pro lepší popis (včetně "upsamplingu") se podívejte např. do [uživatelské příručky](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#resampling). 📖

In [ ]:
widgets.interact(
    lambda x: df.resample(f"{x}M")
    .ffill() # use the last known value (forward filling)
    .loc["1955-01-01":"1956-12-01"]
    .plot(title=f"Perioda {x} měsíců", kind="scatter"),
    x=range(1, 13),
);

### ⬆️ Upsampling
Nebo můžeme počet záznamů zvýšit, tzv. "upsamplovat" na větší frekvenci. Zde můžeme použít např. `lineární interpolaci`, `forward fill` nebo `backward fill`.

In [ ]:
period = ["30D", "21D", "14D", "7D", "1D", "12H", "1H"]
widgets.interact(
    lambda x: df.resample(x)
    .interpolate()
    .loc["1956-09-01":"1956-12-01"]
    .plot(kind="scatter"),
    x=period,
);

## 🥀⛱️🍂❄️ Sezónnost
**Sezónnost** jako taková lze vizualizovat několika způsoby. Typicky můžeme využít čárový graf (angl. line plot).

In [ ]:
multip_decompose.seasonal.plot(title="Seasonality of airline passengers")

> <span style="font-size:200%;"> 🔎 </span>  V grafu vidíme, jak vypadají sezónní hodnoty pro jednotlivé měsíce v  roce.


## 🚧 Transformace

Na závěr zmiňme, že je občas dobrý nápad časovou řadu transformovat. Transformace nám může pomoci časovou řadu například lépe modelovat (např. zmírňuje šum a rozptyl v datech). Mezi často aplikované transformace patří logaritmus, nebo [`box-coxova transformace`](https://www.statisticshowto.com/probability-and-statistics/normal-distributions/box-cox-transformation/). 

In [ ]:
pd.DataFrame(
    {
        "exponential function": pd.Series(
            index=df.index, data=np.exp(np.linspace(4.6, 6.396929655216146, len(df)))
        ),
        "time series": df,
    }
).plot(title="Time series and exponential function")

In [ ]:
pd.DataFrame(
    {
        "linear function": pd.Series(index=df.index, data=np.linspace(4.57, 6.39, len(df))),
        "time series": np.log(df),
    }
).plot(title="Log-transformed time series and linear function")

## 🚔  Autoregresní a Moving-Average modely

⛔️ **POZOR:** Následující část je **vyšší dívčí** (látka magisterského studia) a my si o ní povíme velmi, opravdu velmi letmo. Naším cílem je, abyste pouze tušili, že pro časové řady **existují nějaké modely**, kterými můžete predikovat jejich vývoj, **a grafy** (tj. lollipop graf, o kterém jsme se doteď neučili), které popisují vztah mezi časovou řadou a jejím zpožděním. ⛔️

Časové řady se často používají k modelování (vizte [NI-SCR](https://courses.fit.cvut.cz/MI-SCR/) pro důkladnější výklad). V praxi se často používají **autoregresní (AR) a moving-average modely (MA)** (případně jejich kombinace).

`AR` modely používají jako regresory předchozí hodnoty. Často několik různě zpožděných (*lagged*) měření. 

$$Y_t = \phi_1X_1 +  \phi_2X_2 + \ldots +  \phi_{t+p-1}X_{t+p-1} + \phi_{t+p}X_{t+p} + \epsilon_t, \text{   kde } \epsilon_t \sim \mathcal{N}(0, \sigma^2) $$

`MA` modely fungují na principu náhodné procházky  
$$Y_t = c +  \sum_{k=1}^{q}\theta_{k}\epsilon_{t-k} + \epsilon_t, \text{   kde } \epsilon_i \sim \mathcal{N}(0, \sigma^2) $$

Řády těchto modelů (`p` a `q`) můžeme odhadnout pomocí autocorrelation (`ACF`) a partal-correlation (`PACF`) grafů - nebudeme rozebírat přesně jak, jde nám teď jen o to, abyste věděli, k čemu grafy jsou.

☝️ Jejich vizualizacemi se zabývá např. tento [blogpost](https://towardsdatascience.com/interpreting-acf-and-pacf-plots-for-time-series-forecasting-af0d6db4061c).

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(121)
plot_acf(df, lags=15, ax=ax)
ax.set_ylim([-1, 1.1])
ax = fig.add_subplot(122)
plot_pacf(df, lags=15, ax=ax, method="ywm")
ax.set_ylim([-1, 1.1])
plt.show()

🍭 _"Lízátka"_ (případně "tykadla") v ACF grafu (tzv. "lollipop plot") nám říkají, jak moc je dané zpoždění (angl. lag) (lineárně) korelované s časem $t$. Lízátka v PACF grafu nám říkají, jak moc spolu (lineárně) korelují jednotlivá zpoždění, když je očistíme o vliv mezilehlých zpoždění. V grafech ACF a PACF je navíc vidět modrá oblast. Ta znázorňuje 95% interval spolehlivosti a je ukazatelem prahu významnosti. To znamená, že cokoli uvnitř modré oblasti je statisticky blízké nule a cokoli mimo modrou oblast je statisticky nenulové.

ACF i PACF začínají se zpožděním 0, což je korelace časové řady s ní samotnou, a proto je korelace rovna 1. U grafů pak dále zkoumáme, kolik lízátek je nad nebo pod intervalem spolehlivosti, než další lízátko vstoupí do modré oblasti. (Pomocí toho lze určit model.) Dále v ACF grafu vidíme, že existuje několik autokorelací, které jsou výrazně nenulové. Časová řada tedy není náhodná.

## 🔬 Modelování 
Zkusíme si teď vytvořit model, který je postavený na `AR` a `MA` modelech, ale zvládá zakomponovat také sezónnost a to jak lokální, tak globální. Z modulu `pmdarima.arima` použijeme funkci [`auto_arima`](https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.auto_arima.html), které předáme časovou řadu, periodu měření a informaci o tom, že časová řada obsahuje sezónnost. Funkce nám vrátí optimální model v závislosti na kombinaci informačního kritéria a solveru.

In [ ]:
import pmdarima

In [ ]:
model = pmdarima.arima.auto_arima(df, m=12, seasonal=True, information_criterion="hqic")
model



> <span style="font-size:200%;"> 📝 </span> Výsledný model obsahuje `AR` model s řádem 2, `MA` model s řádem 1, differencování s `k=1` a differencování s `k=12`. 

### 🌦️ Předpověď
Podíváme se, jaké nám bude tento model dávat předpovědi do budoucna.

In [ ]:
from datetime import timedelta
from dateutil.relativedelta import relativedelta

In [ ]:
def forecast(n_months):
    inverse_trans = (lambda x: x) or inverse_trans
    pred, ci = model.predict(n_periods=n_months, return_conf_int=True) # let's use above specified ARIMA

    # create a dataframe with forecast
    df2 = pd.DataFrame(
        data={"forecast": pred, "lb": ci[:, 0], "ub": ci[:, 1]},
        index=pd.DatetimeIndex(
            pd.date_range(
                start=df.index.max(),
                end=df.index.max() + relativedelta(months=n_months),
                freq="M")))

    fig = plt.figure(figsize=(10, 2))
    ax = fig.add_subplot(111)
    ax.plot(df.iloc[-50:], label="data")
    ax.fill_between(df2.index, df2["lb"], df2["ub"], color="b", alpha=0.1, label="confidence interval")
    ax.plot(pred, label="forecast")
    ax.legend()
    fig.suptitle(f"Data with forecast ({n_months} months)")
    plt.show()

In [ ]:
widgets.interact(
    forecast,
    n_months=widgets.IntSlider(
        value=25,
        min=2,
        max=120,
    ),
);

V grafu vidíme modře časovou řadu, oranžovoě předpověď do budoucna a kolem ní konfidenční interval, který nám říká, jak moc jsme si predikcí jistí.



> <span style="font-size:200%;"> 📝 </span> Všimněte si, že čím je predikce v čase vzdálenější od posledního měření, tím menší důvěru model predikci přisuzuje. 


# 🎉 A to je pro tento semestr vše! 🎉

Nezapomeňte prosím **vyplnit Anketu**. Můžete se v ní vyjádřit např. k následujícím otázkám:
- Pokud jste se neúčastnili přednášek fyzicky, koukali jste alespoň na stream/záznamy?
- Vyhovoval vám formát předmětu nebo byste raději měli klasická cvičení v počítačové učebně?
- Líbila se vám jednotlivá témata? Chybělo vám některé téma nebo byste se naopak něčemu věnovali méně?
- Jak hodnotíte jednotlivé domácí úkoly? Bylo pro vás jejich řešení přínosné?

Pomůžete tak předmět zlepšit! 💛

# Děkuji za pozornost! 👏 Hezké svátky! 🎄